In [1]:
#  
""" 
2019.2.7.

ensemble deep learning for time series prediction (reviewing on IEEE JNL)

by kwangsuk.ks.lee at gmail

I proposes a novel ensemble deep learning architecture for time-series forecasting 
of bearing datasets (NASA Prognostics Data Repository) 
by combining two shallow models and two deep models, 
where each model is based on one-dimensional Convolutional Neural Network (1D CNN) 
and Long Short-Term Memory (LSTM). 

The shallow model consists of a few numbers of layers with a number of units, 
which reduces the computational complexity, so that it enables fast learning of the entire ensemble model. 

The deep model with a large number of layers and a small number of units can be 
adequately used for better generalization, and this model is combined with 
a residual network to further reduce overfitting and loss of the entire ensemble model. 

Model 1 of Conv1d Shallow : 256 - 128 - 64  (units)
Model 2 of LSTM  Shallow : 128 - 32 - 128 (units)
Model 3 of Conv1d Deep : 16 - 128 - 16  (1/2 units of shallow model)
Model 4 of lstm Deep : 64 - 16  -  64  (1/2 units of shallow model) 
    
"""

import tensorflow as tf 
import numpy 
import matplotlib.pyplot as plt
import pandas     
from pandas import read_csv, DataFrame, Series  
import math
import keras      
from keras.models import Sequential, Model 
from keras.layers import Dense, Input, Flatten, Add, concatenate 
from keras.layers import Activation, BatchNormalization, regularizers, Dropout

from keras.layers import Average, normalization, TimeDistributed 

#from keras.layers.noise import GaussianNoise
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers import MaxPooling1D, AveragePooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers.merge import concatenate
from keras.layers import LSTM, Bidirectional 
from keras.utils import plot_model  # New 
from os import makedirs 
from sklearn import preprocessing  # New 
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import recall_score, precision_score, f1_score, confusion_matrix  # New 
# http://scikit-learn.org/stable/modules/classes.html 
# http://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau

# fix random seed for reproducibility
# seed = 2458
numpy.random.seed() 


# setting of global variables 
title = '01_01_bearing3_shallow-CL_deep-CL_10e_' # head of file name 
epochs = 10  # Do not modification, plt.plot(... str(str(format(history.first_output_model1_mse[-40,-39-38,-37,-36,-35,-34,-33,-32,-31], ...      
batch_size = 1024 # 128 or 256 or 512 


class LossHistory(keras.callbacks.Callback):   #history = LossHistory()
    def init(self):       # history.init() 
        self.loss = []  # [] 
        self.val_loss = []  # []    
        # model 1   history.history['first_output_model1_loss']
        # self.first_output_model1_loss  = []   # [[], , , ]  # [[],[],[],[]] # [][][][]  # [ , , , ]  # list(, , , ) 
        # self.second_output_model1_loss = []   # [[], , , ]
        self.first_output_model1_mse   = [] 
        self.first_output_model1_mae   = [] 
        self.first_output_model1_mape  = [] 
        self.second_output_model1_mse  = []
        self.second_output_model1_mae  = []
        self.second_output_model1_mape = []
        # self.val_first_output_model1_loss  = [] 
        # self.val_second_output_model1_loss = [] 
        self.val_first_output_model1_mse   = []
        self.val_first_output_model1_mae   = []
        self.val_first_output_model1_mape  = []
        self.val_second_output_model1_mse  = []
        self.val_second_output_model1_mae  = [] # [[], , , ] 
        self.val_second_output_model1_mape = [] # [[], , , ] 
        
        # model 2 
        # self.first_output_model2_loss   = []  # [ ,[], , ]  # [[],[],[],[]] # [][][][]  # [ , , , ]  # list(, , , )   
        # self.second_output_model2_loss  = []  # [ ,[], , ]   
        self.first_output_model2_mse    = []
        self.first_output_model2_mae    = []
        self.first_output_model2_mape   = []
        self.second_output_model2_mse   = []
        self.second_output_model2_mae   = []
        self.second_output_model2_mape  = []
        # self.val_first_output_model2_loss  = [] 
        # self.val_second_output_model2_loss = [] 
        self.val_first_output_model2_mse   = []
        self.val_first_output_model2_mae   = []  
        self.val_first_output_model2_mape  = []  
        self.val_second_output_model2_mse  = []  
        self.val_second_output_model2_mae  = [] # [ ,[], , ]
        self.val_second_output_model2_mape = [] # [ ,[], , ]
        
        # model 3 
        #self.first_output_model3_loss  = [] # [ , , [], ]  # [[],[],[],[]] # [][][][]  # [ , , , ]  # list(, , , ) 
        #self.second_output_model3_loss = [] # [ , , [], ] 
        self.first_output_model3_mse   = []
        self.first_output_model3_mae   = []
        self.first_output_model3_mape  = []
        self.second_output_model3_mse  = []
        self.second_output_model3_mae  = []
        self.second_output_model3_mape = []
        #self.val_first_output_model3_loss = []  
        #self.val_second_output_model3_loss = []
        self.val_first_output_model3_mse   = []
        self.val_first_output_model3_mae   = []
        self.val_first_output_model3_mape  = []
        self.val_second_output_model3_mse  = []
        self.val_second_output_model3_mae  = [] # [ , , [], ] 
        self.val_second_output_model3_mape = [] # [ , , [], ] 
        
        # model 4 
        #self.first_output_model4_loss = []  #  [ , , , []]  # [[],[],[],[]] # [][][][]  # [ , , , ]  # list(, , , ) 
        #self.second_output_model4_loss = [] #  [ , , , []] 
        self.first_output_model4_mse = [] 
        self.first_output_model4_mae = []
        self.first_output_model4_mape = []
        self.second_output_model4_mse = []
        self.second_output_model4_mae = []
        self.second_output_model4_mape = []
                                          
        #self.val_first_output_model4_loss = [] 
        #self.val_second_output_model4_loss = []                 
        self.val_first_output_model4_mse = []
        self.val_first_output_model4_mae = []
        self.val_first_output_model4_mape = []
        self.val_second_output_model4_mse = []
        self.val_second_output_model4_mae = []  # [ , , ,[]]
        self.val_second_output_model4_mape = [] # [ , , ,[]] 

    def on_epoch_end(self, batch, logs={}):  #def on_epoch_end(self, batch, logs={}):
        # self.loss.append(logs.get('loss'))  
        # self.val_loss.append(logs.get('val_loss'))  
        # self.loss.append(logs['loss'])
        # self.val_loss.append(logs['val_loss'])
        self.loss.append(logs.get("loss")) ### 
        self.val_loss.append(logs.get("val_loss")) ###  
        # model 1 
        #self.first_output_model1_loss[0].append(logs.get("first_output_model1_loss")) 
        #self.second_output_model1_loss[0].append(logs.get("second_output_model1_loss"))
        self.first_output_model1_mse.append(logs.get("first_output_model1_mean_squared_error")) 
        self.first_output_model1_mae.append(logs.get("first_output_model1_mean_absolute_error"))
        self.first_output_model1_mape.append(logs.get("first_output_model1_mean_absolute_percentage_error"))
        self.second_output_model1_mse.append(logs.get("second_output_model1_mean_squared_error"))
        self.second_output_model1_mae.append(logs.get("second_output_model1_mean_absolute_error")) 
        self.second_output_model1_mape.append(logs.get("second_output_model1_mean_absolute_percentage_error"))
        
        #self.val_first_output_model1_loss[0].append(logs.get("val_first_output_model1_loss")) 
        #self.val_second_output_model1_loss[0].append(logs.get("val_second_output_model1_loss")) 
        self.val_first_output_model1_mse.append(logs.get("val_first_output_model1_mean_squared_error")) 
        self.val_first_output_model1_mae.append(logs.get("val_first_output_model1_mean_absolute_error")) 
        self.val_first_output_model1_mape.append(logs.get("val_first_output_model1_mean_absolute_percentage_error")) 
        self.val_second_output_model1_mse.append(logs.get("val_second_output_model1_mean_squared_error")) 
        self.val_second_output_model1_mae.append(logs.get("val_second_output_model1_mean_absolute_error")) 
        self.val_second_output_model1_mape.append(logs.get("val_second_output_model1_mean_absolute_percentage_error")) 
        
        # model 2 
        #self.first_output_model2_loss[1].append(logs.get("first_output_model2_loss")) 
        #self.second_output_model2_loss[1].append(logs.get("second_output_model2_loss"))                               
        self.first_output_model2_mse.append(logs.get("first_output_model2_mean_squared_error"))  
        self.first_output_model2_mae.append(logs.get("first_output_model2_mean_absolute_error")) 
        self.first_output_model2_mape.append(logs.get("first_output_model2_mean_absolute_percentage_error")) 
        self.second_output_model2_mse.append(logs.get("second_output_model2_mean_squared_error")) 
        self.second_output_model2_mae.append(logs.get("second_output_model2_mean_absolute_error")) 
        self.second_output_model2_mape.append(logs.get("second_output_model2_mean_absolute_percentage_error")) 
        
        #self.val_first_output_model2_loss[1].append(logs.get("val_first_output_model2_loss")) 
        #self.val_second_output_model2_loss[1].append(logs.get("val_second_output_model2_loss")) 
        self.val_first_output_model2_mse.append(logs.get("val_first_output_model2_mean_squared_error")) 
        self.val_first_output_model2_mae.append(logs.get("val_first_output_model2_mean_absolute_error")) 
        self.val_first_output_model2_mape.append(logs.get("val_first_output_model2_mean_absolute_percentage_error")) 
        self.val_second_output_model2_mse.append(logs.get("val_second_output_model2_mean_squared_error")) 
        self.val_second_output_model2_mae.append(logs.get("val_second_output_model2_mean_absolute_error")) 
        self.val_second_output_model2_mape.append(logs.get("val_second_output_model2_mean_absolute_percentage_error")) 
        
        # model 3 
        #self.first_output_model3_loss[2].append(logs.get("first_output_model3_loss")) 
        #self.second_output_model3_loss[2].append(logs.get("second_output_model3_loss"))                             
        self.first_output_model3_mse.append(logs.get("first_output_model3_mean_squared_error")) 
        self.first_output_model3_mae.append(logs.get("first_output_model3_mean_absolute_error")) 
        self.first_output_model3_mape.append(logs.get("first_output_model3_mean_absolute_percentage_error")) 
        self.second_output_model3_mse.append(logs.get("second_output_model3_mean_squared_error")) 
        self.second_output_model3_mae.append(logs.get("second_output_model3_mean_absolute_error")) 
        self.second_output_model3_mape.append(logs.get("second_output_model3_mean_absolute_percentage_error"))
        
        #self.val_first_output_model3_loss[2].append(logs.get("val_first_output_model3_loss")) 
        #self.val_second_output_model3_loss[2].append(logs.get("val_second_output_model3_loss")) 
        self.val_first_output_model3_mse.append(logs.get("val_first_output_model3_mean_squared_error")) 
        self.val_first_output_model3_mae.append(logs.get("val_first_output_model3_mean_absolute_error")) 
        self.val_first_output_model3_mape.append(logs.get("val_first_output_model3_mean_absolute_percentage_error")) 
        self.val_second_output_model3_mse.append(logs.get("val_second_output_model3_mean_squared_error")) 
        self.val_second_output_model3_mae.append(logs.get("val_second_output_model3_mean_absolute_error"))
        self.val_second_output_model3_mape.append(logs.get("val_second_output_model3_mean_absolute_percentage_error")) 
        
        # model 4 
        #self.first_output_model4_loss[3].append(logs.get("first_output_model4_loss")) 
        #self.second_output_model4_loss[3].append(logs.get("second_output_model4_loss"))                                 
        self.first_output_model4_mse.append(logs.get("first_output_model4_mean_squared_error")) 
        self.first_output_model4_mae.append(logs.get("first_output_model4_mean_absolute_error")) 
        self.first_output_model4_mape.append(logs.get("first_output_model4_mean_absolute_percentage_error")) 
        self.second_output_model4_mse.append(logs.get("second_output_model4_mean_squared_error")) 
        self.second_output_model4_mae.append(logs.get("second_output_model4_mean_absolute_error")) 
        self.second_output_model4_mape.append(logs.get("second_output_model4_mean_absolute_percentage_error"))    
        
        #self.val_first_output_model4_loss[3].append(logs.get("val_first_output_model4_loss")) 
        #self.val_second_output_model4_loss[3].append(logs.get("val_second_output_model4_loss")) 
        self.val_first_output_model4_mse.append(logs.get("val_first_output_model4_mean_squared_error")) 
        self.val_first_output_model4_mae.append(logs.get("val_first_output_model4_mean_absolute_error")) 
        self.val_first_output_model4_mape.append(logs.get("val_first_output_model4_mean_absolute_percentage_error")) 
        self.val_second_output_model4_mse.append(logs.get("val_second_output_model4_mean_squared_error")) 
        self.val_second_output_model4_mae.append(logs.get("val_second_output_model4_mean_absolute_error")) 
        self.val_second_output_model4_mape.append(logs.get("val_second_output_model4_mean_absolute_percentage_error")) 
        
# end of the class 

# convert an array of values into a dataset matrix
def create_dataset(dataset, timesteps): #기존def create_dataset(dataset, timesteps=1): 여기서 -1삭제
    dataX, dataY = [], []
    for i in range(len(dataset)-timesteps-1):
        a = dataset[i:(i+timesteps), 0]
        dataX.append(a)
        dataY.append(dataset[i + timesteps, 0])
    return numpy.array(dataX), numpy.array(dataY)


# load the dataset

# TRAIN of X-xais Bearing 3 
t = '00_Train_2003.11.25.12.17_14.07_12sets.csv'
df_train_1 = pandas.read_csv(t, header=None, sep='	', usecols=[4], engine='python') # X-xais of bearing 3 
dataset_train_1 = df_train_1.values  # 
dataset_train_1 = dataset_train_1.astype('float32')
print(' dataset_train_1.shape == ', dataset_train_1.shape)

# TRAIN of Y-xais Bearing 3 
df_train_2 = pandas.read_csv(t, header=None, sep='	', usecols=[5], engine='python') # Y-xais of bearing 3
dataset_train_2 = df_train_2.values  # 
dataset_train_2 = dataset_train_2.astype('float32')
print(' dataset_train_2.shape == ', dataset_train_2.shape)

# Test of X-axis Bearing 3
t1 = '01_Test_2003.11.25.14.17_16.07_12sets.csv' 
df_test_1 = pandas.read_csv(t1, header=None, sep='	', usecols=[4], engine='python') # Test of X-axis Bearing 3
dataset_test_1 = df_test_1.values  # Prediction Target(label) Data
dataset_test_1 = dataset_test_1.astype('float32')
print(' dataset_test_1.shape == ', dataset_test_1.shape)

# Test of Y-axis Bearing 3  
df_test_2 = pandas.read_csv(t1, header=None, sep='	', usecols=[5], engine='python') # Test of Y-axis Bearing 3  
dataset_test_2 = df_test_2.values  # Prediction Target(label) Data
dataset_test_2 = dataset_test_2.astype('float32')
print(' dataset_test_2.shape == ', dataset_test_2.shape)




# normalize the dataset ---------------- # 18.2.25. 
scaler1 = MinMaxScaler(feature_range=(0, 1))  
dataset_train_1 = scaler1.fit_transform(dataset_train_1)

scaler2 = MinMaxScaler(feature_range=(0, 1)) # 18.3.16.
dataset_train_2 = scaler2.fit_transform(dataset_train_2)

scaler3 = MinMaxScaler(feature_range=(0, 1)) 
dataset_test_1 = scaler3.fit_transform(dataset_test_1) # TEST of X-xais of bearing 1

scaler4 = MinMaxScaler(feature_range=(0, 1)) 
dataset_test_2 = scaler4.fit_transform(dataset_test_2) # TEST of Y-xais of bearing 1

print('-----------------------------------------')
print(' dataset_train_1.shape == ', dataset_train_1.shape)
print(' dataset_train_2.shape == ', dataset_train_2.shape)

print(' dataset_test_1.shape == ', dataset_test_1.shape) # dataset_test_1  
print(' dataset_test_2.shape == ', dataset_test_2.shape) # dataset_test_2
print('-----------------------------------------')
dataset_train_1_mean = numpy.mean(dataset_train_1) # mean  
print('mean of dataset_train_1 == ', dataset_train_1_mean)
dataset_train_1_var = numpy.var(dataset_train_1)  # variance, The variance is the average of the squared deviations from the mean, i.e., var = mean(abs(x - x.mean())**2). 
print('variance of dataset_train_1 == ', dataset_train_1_var)
dataset_train_1_std = numpy.std(dataset_train_1) # std, The standard deviation is the square root of the average of the squared deviations from the mean, i.e., std = sqrt(mean(abs(x - x.mean())**2)) 
print('standard deviation of dataset_train_1 == ', dataset_train_1_std)
print('-----------------------------------------')
dataset_train_2_mean = numpy.mean(dataset_train_2) # mean  
print('mean of dataset_train_1 == ', dataset_train_2_mean)
dataset_train_2_var = numpy.var(dataset_train_2)  # variance  
print('variance of dataset_train_2 == ', dataset_train_2_var)
dataset_train_2_std = numpy.std(dataset_train_2) # standard deviation 
print('standard deviation of dataset_train_2 == ', dataset_train_2_std)

print('-----------------------------------------')
dataset_test_1_mean = numpy.mean(dataset_test_1) # mean    ... # TEST of X-xaix of bearing 1
print('mean of dataset_test_1 == ', dataset_test_1_mean)
dataset_test_1_var = numpy.var(dataset_test_1)  # variance
print('variance of dataset_test_1 == ', dataset_test_1_var)
dataset_test_1_std = numpy.std(dataset_test_1) # standard deviation
print('standard deviation of dataset_test_1 == ', dataset_test_1_std)
print('-----------------------------------------')
dataset_test_2_mean = numpy.mean(dataset_test_2) # mean    ... # TEST of Y-xaix of bearing 1
print('mean of dataset_test_2 == ', dataset_test_2_mean)
dataset_test_2_var = numpy.var(dataset_test_2)  # variance
print('variance of dataset_test_2 == ', dataset_test_2_var)
dataset_test_2_std = numpy.std(dataset_test_2) # standard deviation
print('standard deviation of dataset_test_2 == ', dataset_test_2_std)
print('-----------------------------------------')

# writing var, std to file 
file = open(title+"var_std_train.txt", 'w') 
file.write("\n# variance, standard deviation of train and test sets ")
file.write("\n mean of dataset_train_1 == ")
file.write(str(dataset_train_1_mean))  #write() argument must be str, not numpy.float32
file.write("\n variance of dataset_train_1 == ")
file.write(str(dataset_train_1_var))
file.write("\n standard deviation of dataset_train_1 == ")
file.write(str(dataset_train_1_std))
file.write("\n----------------------------------------")
file.write("\n mean of dataset_train_1 == ")
file.write(str(dataset_train_2_mean))
file.write("\n variance of dataset_train_2 == ")
file.write(str(dataset_train_2_var))
file.write("\n standard deviation of dataset_train_2 == ")
file.write(str(dataset_train_2_std))
file.write("\n-----------------------------------------")

file.write("\n mean of dataset_test_1 == ")
file.write(str(dataset_test_1_mean))        #  ...  or  X-xaix of bearing 1
file.write("\n variance of dataset_test_1 == ")
file.write(str(dataset_test_1_var)) 
file.write("\n standard deviation of dataset_test_1 == ")
file.write(str(dataset_test_1_std))
file.write("\n-----------------------------------------")
file.write("\n mean of dataset_test_2 == ")
file.write(str(dataset_test_2_mean))        # ...  or Y-xaix of bearing 1
file.write("\n variance of dataset_test_2 == ")
file.write(str(dataset_test_2_var)) 
file.write("\n standard deviation of dataset_test_2 == ")
file.write(str(dataset_test_2_std)) 
file.write("-----------------------------------------")
file.close() 

# split into train and test sets ----------------
train_1_size = int(len(dataset_train_1)) 
train_2_size = int(len(dataset_train_2))

test_1_size = int(len(dataset_test_1)) #  TEST of X-xaix of bearing 1
test_2_size = int(len(dataset_test_2)) #  TEST of Y-xaix of bearing 1

print('-----------------------------------------')
train_1 = dataset_train_1[0:train_1_size,:]  
train_2 = dataset_train_2[0:train_2_size,:]

test_1 = dataset_test_1[0:test_1_size,:] # TEST of X-xaix of bearing 1
test_2 = dataset_test_2[0:test_2_size,:] # TEST of Y-xaix of bearing 1

print('length of train_1  == ', int(len(train_1)))
print('length of train_2 == ', int(len(train_2))) 

print('length of test_1 == ', int(len(test_1))) # X-xaix of bearing 1
print('length of test_2 == ', int(len(test_2))) # Y-xaix of bearing 1 
print(' Lets start ~!!! ') 
print('-----------------------------------------')

features = 1
timesteps = 29 #  

trainX_1, trainY_1 = create_dataset(train_1, timesteps) # Train X-axis Bearing 3 all dataset for shallow mosels 
trainX_2, trainY_2 = create_dataset(train_2, timesteps) # Train Y-axis Bearing 3 all dataset for shallow models  

testX_1, testY_1 = create_dataset(test_1, timesteps) # TEST of X-xaix of bearing 1
testX_2, testY_2 = create_dataset(test_2, timesteps) # TEST Y-xaix of bearing 1 


# reshape input to be [samples, time steps, features] 
trainX_1 = numpy.reshape(trainX_1, (trainX_1.shape[0], trainX_1.shape[1], 1))
testX_1 = numpy.reshape(testX_1, (testX_1.shape[0], testX_1.shape[1], 1))

trainX_2 = numpy.reshape(trainX_2, (trainX_2.shape[0], trainX_2.shape[1], 1))
testX_2 = numpy.reshape(testX_2, (testX_2.shape[0], testX_2.shape[1], 1))
print('-----------------------------------')  
print(' train_X.shape = ', trainX_1.shape) # train_X.shape =  (245730, 29, 1)
print(' train_Y.shape = ', trainY_1.shape) # train_Y.shape =  (245730,)
print(' train_X.shape = ', trainX_2.shape) # train_X.shape =  (245730, 29, 1)
print(' train_Y.shape = ', trainY_2.shape) # train_Y.shape =  (245730,)

print(' test_X.shape == ', testX_1.shape)  #  test_X.shape ==  (245730, 29, 1)  feature 
print(' test_Y.shape == ', testY_1.shape)  #  test_Y.shape ==  (245730,)  target 
print(' test_X.shape == ', testX_2.shape)  #  test_X.shape ==  (245730, 29, 1) feature 
print(' test_Y.shape == ', testY_2.shape)  #  test_Y.shape ==  (245730,)  target 
print('-----------------------------------') 
# testX_1 : testX 1 feature 
# testY_1 : testX 1 target(label)

# testX_2 : testX 2 feature 
# testY_2 : testx 2 target(label) 

######################################### 
#batch_size = 32

model_input_1 = Input(shape=(timesteps, features), name='first_input')  # (29, 1)  X-aix of bearing 3 

model_input_2 = Input(shape=(timesteps, features), name='second_input')  # (29, 1) y-aix of bearing 3 

import keras.backend.tensorflow_backend as K 
with K.tf.device('/gpu:0'):   #cpu:/0
     
    def model1_def(model_input_1, model_input_2):  
        
        inputs1_model1 = model_input_1
        
        conv_11 = Conv1D(filters=128, kernel_size=1, strides=1)(inputs1_model1)
        #conv_11 = Conv1D(filters=64, kernel_size=1, strides=1, kernel_regularizer=regularizers.l2(0.001))(inputs1_model1)#(inputs1) 
        """conv_11_bn = BatchNormalization()(conv_11) # 
        conv_11_bn_act = Activation('relu')(conv_11_bn)
        conv_11_out = Dropout(0.1)(conv_11_bn_act) # (29,128)"""
                                
        inputs2_model1 = model_input_2
        
        conv_21 = Conv1D(filters=128, kernel_size=1, strides=1)(inputs2_model1) 
        # conv_21 = Conv1D(filters=64, kernel_size=1, strides=1, kernel_regularizer=regularizers.l2(0.001))(inputs2_model1)#(inputs2) 
        """conv_21_bn = BatchNormalization()(conv_21) # 
        conv_21_bn_act = Activation('relu')(conv_21_bn)
        conv_21_out = Dropout(0.1)(conv_21_bn_act) # (29,128)"""
        
        concatenated = concatenate([conv_11, conv_21], axis=-1)  
        
        conv_31 = Conv1D(filters=128, kernel_size=1, strides=1)(concatenated) 
        #conv_31 = Conv1D(filters=32, kernel_size=1, strides=1, kernel_regularizer=regularizers.l2(0.001))(concatenated) 
        
        conv_32 = Conv1D(filters=64, kernel_size=1, strides=1)(conv_31) 
        #conv_32 = Conv1D(filters=128, kernel_size=1, strides=1, kernel_regularizer=regularizers.l2(0.001))(conv_31) 
        
        ls_11 = (LSTM(64, dropout=0.1, recurrent_dropout=0.1))(conv_32)
        
        # LAST POINT, Do Not residual connection at this last layer ~!
        out_1 = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(ls_11)  
        
        outputs1_model1 = Dense(1, activation='relu', name = 'first_output_model1')(out_1)  # name
        outputs2_model1 = Dense(1, activation='relu', name = 'second_output_model1')(out_1) # name
        
        model1 = Model(inputs=[inputs1_model1, inputs2_model1], 
                       outputs=[outputs1_model1, outputs2_model1], name='model1_conv_shallow') # name  
        
        return model1   
        
   
    def model2_def(model_input_1, model_input_2):  
        
        inputs1_model2 = model_input_1
        
        ls_11 = (LSTM(64, return_sequences=True, dropout=0.1))(inputs1_model2)    
        # conv_11 = Conv1D(filters=64, kernel_size=1, strides=1, kernel_regularizer=regularizers.l2(0.001))(inputs1_model2)    
        """     conv_11_bn = BatchNormalization()(conv_11) 
        conv_11_bn_act = Activation('relu')(conv_11_bn)
        conv_11_out = Dropout(0.1)(conv_11_bn_act) # (29,128)"""
            
        inputs2_model2 = model_input_2
        
        ls_21 = (LSTM(64, return_sequences=True, dropout=0.1))(inputs2_model2)#(inputs2) 
        #conv_21 = Conv1D(filters=64, kernel_size=1, strides=1, kernel_regularizer=regularizers.l2(0.001))(inputs2_model2)#(inputs2) 
        """conv_21_bn = BatchNormalization()(conv_21) 
        conv_21_bn_act = Activation('relu')(conv_21_bn)
        conv_21_out = Dropout(0.1)(conv_21_bn_act) # (29,128)"""
            
        concatenated = concatenate([ls_11, ls_21], axis=-1)  # 128 <== 64 + 64
        
        ls_31 = (LSTM(32, return_sequences=True, dropout=0.1))(concatenated)    
        ls_32 = (LSTM(128, dropout=0.1))(ls_31) #last lstm layer, remove..., return_sequences=True. 
        #ls_31 = (LSTM(32, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(concatenated)    
        #ls_32 = (LSTM(128, dropout=0.1, recurrent_dropout=0.1))(ls_31) #last lstm layer, remove..., return_sequences=True. 

        # LAST POINT, Do Not residual connection at this last layer ~!
        out_1 = Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001))(ls_32)  
        
        outputs1_model2 = Dense(1, activation='relu', name = 'first_output_model2')(out_1)  # name
        outputs2_model2 = Dense(1, activation='relu', name = 'second_output_model2')(out_1) # name

        model2 = Model(inputs=[inputs1_model2, inputs2_model2], 
                       outputs=[outputs1_model2, outputs2_model2], name='model2_lstm_shallow') # name 

        return model2       
        

    def model3_def(model_input_1, model_input_2):  
        
        inputs1_model3 = model_input_1
        
        conv_11 = Conv1D(filters=8, kernel_size=1, strides=1)(inputs1_model3) #(inputs1) 
        #conv_11 = Conv1D(filters=64, kernel_size=1, strides=1, kernel_regularizer=regularizers.l2(0.001))(inputs1_model3) #(inputs1) 
        """   conv_11_bn = BatchNormalization()(conv_11) 
        conv_11_bn_act = Activation('relu')(conv_11_bn)
        conv_11_out = Dropout(0.1)(conv_11_bn_act) # (29,128)"""
            
        inputs2_model3 = model_input_2
        
        conv_21 = Conv1D(filters=8, kernel_size=1, strides=1)(inputs2_model3) #(inputs2) 
        #conv_21 = Conv1D(filters=64, kernel_size=1, strides=1, kernel_regularizer=regularizers.l2(0.001))(inputs2_model3) #(inputs2) 
        """   conv_21_bn = BatchNormalization()(conv_21) 
        conv_21_bn_act = Activation('relu')(conv_21_bn)
        conv_21_out = Dropout(0.1)(conv_21_bn_act) # (29,128)""" 
            
        concatenated = concatenate([conv_11, conv_21], axis=-1)  # 16 = 8+8 
        
        conv_31 = Conv1D(filters=16, kernel_size=1, strides=1)(concatenated)  
        conv_32 = Conv1D(filters=128, kernel_size=1, strides=1)(conv_31)  
        conv_33 = Conv1D(filters=16, kernel_size=1, strides=1)(conv_32)  
        #conv_31 = Conv1D(filters=32, kernel_size=1, strides=1, kernel_regularizer=regularizers.l2(0.001))(concatenated) 
    
        residual_1 = keras.layers.Add()([conv_33, concatenated])  # 16, 16 
        
 
        conv_34 = Conv1D(filters=16, kernel_size=1, strides=1)(residual_1) 
        #conv_34_out = Dropout(0.1)(conv_32)
        conv_35 = Conv1D(filters=128, kernel_size=1, strides=1)(conv_34)   
        #conv_32 = Conv1D(filters=128, kernel_size=1, strides=1, kernel_regularizer=regularizers.l2(0.001))(conv_31) 
        conv_36 = Conv1D(filters=16, kernel_size=1, strides=1)(conv_35) 
        #conv_34_out = Dropout(0.1)(conv_32)

        residual_2 = keras.layers.Add()([conv_36, residual_1])  # 16, 16
        
   
        conv_37 = Conv1D(filters=16, kernel_size=1, strides=1)(residual_2) 
        #conv_34_out = Dropout(0.1)(conv_32)
        conv_38 = Conv1D(filters=128, kernel_size=1, strides=1)(conv_37)   
        #conv_32 = Conv1D(filters=128, kernel_size=1, strides=1, kernel_regularizer=regularizers.l2(0.001))(conv_31) 
        conv_39 = Conv1D(filters=16, kernel_size=1, strides=1)(conv_38) 
        #conv_34_out = Dropout(0.1)(conv_32)
        ls_11 = (LSTM(64, dropout=0.1, recurrent_dropout=0.1))(conv_39)
        # LAST POINT, Do Not residual connection at this last layer ~!

        out_1 = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(ls_11) # regularizers
        
        outputs1_model3 = Dense(1, activation='relu', name = 'first_output_model3')(out_1)  # name
        outputs2_model3 = Dense(1, activation='relu', name = 'second_output_model3')(out_1) # name

        model3 = Model(inputs=[inputs1_model3, inputs2_model3], 
                       outputs=[outputs1_model3, outputs2_model3], name='model3_conv_deep') # name
        
        return model3  
        
        
    def model4_def(model_input_1, model_input_2):  

        inputs1_model4 = model_input_1

        ls_11 = (LSTM(32, return_sequences=True))(inputs1_model4)#(inputs1) 
        #ls_11 = (LSTM(16, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(inputs1_model4)#(inputs1) 
        """ls_11_bn = BatchNormalization()(ls_11) # 배치놈은 반드시 활성화함수 이전에 사용
        ls_11_bn_act = Activation('relu')(ls_11_bn)
        ls_11_out = Dropout(0.1)(ls_11_bn_act) # (29,64)"""
        
        inputs2_model4 = model_input_2
        
        ls_21 = (LSTM(32, return_sequences=True))(inputs2_model4)#(inputs2) 
        #ls_21 = (LSTM(16, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(inputs2_model4)#(inputs2) 
        """ls_21_bn = BatchNormalization()(ls_21) # 배치놈은 반드시 활성화함수 이전에 사용
        ls_21_bn_act = Activation('relu')(ls_21_bn)
        ls_21_out = Dropout(0.1)(ls_21_bn_act) # (29,64)"""

        concatenated = concatenate([ls_11, ls_21], axis=-1)  # 64 = 32+32 
               
        # 2~4th Layer --- 64 - 16 - 64
        ls_31 = (LSTM(64, return_sequences=True, dropout=0.1))(concatenated)    
        ls_32 = (LSTM(16, return_sequences=True, dropout=0.1))(ls_31) 
        ls_33 = (LSTM(64, return_sequences=True, dropout=0.1))(ls_32) 
        #ls_31 = (LSTM(32, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(concatenated)    
        #ls_32 = (LSTM(128, dropout=0.1, recurrent_dropout=0.1))(ls_31) #last lstm layer, remove..., return_sequences=True. 
        
        # ever 2~3layers 
        residual_1 = keras.layers.Add()([ls_33, concatenated])  # 16, 16 
        
        # 5~7th Layer --- 64 - 16 - 64
        ls_34 = (LSTM(64, return_sequences=True, dropout=0.1))(residual_1)    
        ls_35 = (LSTM(16, return_sequences=True, dropout=0.1))(ls_34) 
        ls_36 = (LSTM(64, return_sequences=True, dropout=0.1))(ls_35) 
        
        # ever 2~3layers 
        residual_2 = keras.layers.Add()([ls_36, residual_1])  # 16, 16 
        
        # 8~10th Layer --- 64 - 16 - 64
        ls_37 = (LSTM(64, return_sequences=True, dropout=0.1))(residual_2)   
        ls_38 = (LSTM(16, return_sequences=True, dropout=0.1))(ls_37)   
        ls_39 = (LSTM(64, dropout=0.1))(ls_38) #last lstm layer, remove..., return_sequences=True. 

        # =========== # LAST POINT, Do Not residual connection at this last layer ~!
        # 4th Layer 
        out_1 = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(ls_39)   
        
        outputs1_model4 = Dense(1, activation='relu', name = 'first_output_model4')(out_1)  # name
        outputs2_model4 = Dense(1, activation='relu', name = 'second_output_model4')(out_1) # name

        model4 = Model(inputs=[inputs1_model4, inputs2_model4], 
                       outputs=[outputs1_model4, outputs2_model4], name = 'model4_lstm_deep') # name  
          
        return model4   
    ####################################################
    def compile_and_train(model, epochs):
        
        optimizer = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0, amsgrad=False)
        
        model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse', 'mae', 'mape'])    

        #epochs=10   
        #batch_size = 32  
        saveweightfile = title + model.name + '.{epoch:02d}-_{loss:.2f}.hdf5'  # title # head of file name 
        checkpoint = ModelCheckpoint(saveweightfile, monitor='loss', verbose=1,
                                     save_weights_only=True, save_best_only=True, mode='auto', period=1)

        reducelr = ReduceLROnPlateau(monitor='val_loss', factor = 0.1, patience=1) #에포크 1동안 좋아지지 않으면 호출 

        earlystop = EarlyStopping(monitor='val_loss', patience=3) #에포크 4(3이상) 좋아지지 않으면 스탑 

        csv_logger = CSVLogger(title + 'log.csv', append=True, separator='\n')  # separator=';', separator='\t'
        
        #history = LossHistory()
        #history.init() 
        #print ('Total epochs :  ' + str(i+1) + '  of ' + str(epochs))  # i = 0,1,2...
        model.fit([trainX_1, trainX_2], [trainY_1, trainY_2],
                  epochs=epochs,
                  batch_size=batch_size,
                  shuffle=False,
                  validation_data=([testX_1, testX_2], [testY_1, testY_2]), 
                  callbacks=[history, csv_logger, checkpoint, reducelr, earlystop]) 
        
        savemodelfile = title + model.name + str(epochs) + '.h5' # title # head of file name 
        model.save(savemodelfile)
        print('> Saved trained model : %s' % savemodelfile)
        return history   #return " Train complete.\n"
    
    
    #-------------------------------------------------------------------
    model1_model = model1_def(model_input_1, model_input_2) #  New model 1 ... Conv Shallow   
    model2_model = model2_def(model_input_1, model_input_2) #  New model 2 ... LSTM Shallow
    model3_model = model3_def(model_input_1, model_input_2) #  New model 3 ... ResNet of conv1d
    model4_model = model4_def(model_input_1, model_input_2) #  New model 4 ... ResNet of LSTM 
        
    history = LossHistory() # 2019.12.15.
    history.init()          # 2019.12.15.
         
    _ = compile_and_train(model1_model, epochs) # return history
    _ = compile_and_train(model2_model, epochs) # return history
    
    _ = compile_and_train(model3_model, epochs)  
    _ = compile_and_train(model4_model, epochs)  
        # print('_',_)  # output is _ <keras.callbacks.History object       


    def ensemble(models, model_input_1, model_input_2):
        out1 = [model.outputs[0] for model in models]  # model.output[] , model = keras.Model 
        out2 = [model.outputs[1] for model in models]
        # for i in range(int(X.shape[0])) 
        print('--------------------------------')
        print('out1... = ', out1)  # tensor 4 items out, [<tf.Tensor 'first_output_model1/Relu:0' shape=(?, 1) >, <tf.Tensor 'first_output_model2/Relu:0' shape=(?, 1) >, <tf.Tensor 'first_output_model3/Relu:0' shape=(?, 1) >, <tf.Tensor 'first_output_model4/Relu:0' shape=(?, 1) >]
        print('--------------------------------')
        print('out2... = ', out2)  # tensor 4 items out, [<tf.Tensor 'second_output_model1/Relu:0' shape=(?, 1) , <tf.Tensor 'second_output_model2/Relu:0' shape=(?, 1) >, <tf.Tensor 'second_output_model3/Relu:0' shape=(?, 1) >, <tf.Tensor 'second_output_model4/Relu:0' shape=(?, 1) >]
        print('--------------------------------')
        output1_avr = Average()(out1) 
        output2_avr = Average()(out2) 
        print('--------------------------------')
        print('output1_var... = ', output1_avr) # tensor 1 item out, output1_var...= Tensor("average_1/truediv:0", shape=(?, 1), dtype=float32, device=/device:GPU:0)  
        print('--------------------------------')
        print('output2_var... = ', output2_avr) # tensor 1 item out, output2_var...= Tensor("average_2/truediv:0", shape=(?, 1), dtype=float32, device=/device:GPU:0)
        print('--------------------------------')
        model = Model(inputs=[model_input_1, model_input_2], 
                      outputs=[output1_avr, output2_avr], 
                      name='ensemble')       
        return model
    
    
    models = [model1_model, model2_model, model3_model, model4_model] 
    ###  model 1 ~ 4  = Model(inputs=[inputs1_, inputs2_], outputs=[outputs1_, outputs2_] )
    # models_group2 = [model3_model, model4_model]
    # models_group3 = [model1_model, model2_model, model3_model]
    # models_group4 = [model1_model, model2_model, model3_model, model4_model]

    ensemble_model = ensemble(models, model_input_1, model_input_2)

    
    # make predictions,    
    print(' Start Prediction ... ')   
    print(' Train X_1 (bearing 3 x), Train X_2 (bearing 3 y) Predicting...') 
    trainPredict_1, trainPredict_2 = ensemble_model.predict([trainX_1, trainX_2], batch_size=batch_size, verbose = 1) 
    print(' Test X_1 (bearing 3 x), Test X_2 (bearing 3 y) Predicting...')  
    testPredict_1, testPredict_2 = ensemble_model.predict([testX_1, testX_2], batch_size=batch_size, verbose = 1) 
    print(' Prediction complete ...')    
    
# --------------------------------------------------------------        
print('trainPredict_1 = model.predict([trainX_1, ], ) =', trainPredict_1)
print('trainPredict_1.shape = ', trainPredict_1.shape)
print('testPredict_1 = model.predict([testX_1, ], ) = ', testPredict_1)
print('testPredict_1.shape = ', testPredict_1.shape)
# --------------------------------------------------------------  

# invert predictions  
trainPredict_1 = scaler1.inverse_transform(numpy.array(trainPredict_1)) #trainX_1 or  (numpy.array(trainPredict_1)) 
trainY_1 = scaler1.inverse_transform(numpy.array([trainY_1,])) #trainY_1 or (numpy.array([trainY_1,]))
print('scaler.inverse_transform(numpy.array(trainPredic_1)) = ', trainPredict_1)
print('scaler.inverse_transform(numpy.array([trainY_1])) = ', trainY_1)

testPredict_1 = scaler3.inverse_transform(numpy.array(testPredict_1)) #testX_1  
testY_1 = scaler3.inverse_transform(numpy.array([testY_1])) #testY_1 or (numpy.array([testY_1,]))
print('scaler.inverse_transform(numpy.array(testPredic_1)) = ', testPredict_1)
print('scaler.inverse_transform(numpy.array([testY_1])) = ', testY_1)

trainPredict_2 = scaler2.inverse_transform(numpy.array(trainPredict_2)) #trainX_2
trainY_2 = scaler2.inverse_transform(numpy.array([trainY_2])) #testY_2 or (numpy.array([testY_2]))
print('scaler.inverse_transform(numpy.array(trainPredic_2)) = ', trainPredict_2)
print('scaler.inverse_transform(numpy.array(trainY_2)) = ', trainY_2)

testPredict_2 = scaler4.inverse_transform(numpy.array(testPredict_2)) #testX_2
testY_2 = scaler4.inverse_transform(numpy.array([testY_2])) #testY_2 or (numpy.array([testY_2]))  
print('scaler.inverse_transform(numpy.array(testPredic_2)) = ', testPredict_2)
print('scaler.inverse_transform(numpy.array(testY_1)) = ', testY_2)
        
# calculate root mean squared error of prediction  

print('=========================================')
trainScore_1 = math.sqrt(mean_squared_error(trainY_1[0], trainPredict_1[:,0])) #trainY_1, trainX_1
print('Train_1 Score: %.5f RMSE' % (trainScore_1))  

testScore_1 = math.sqrt(mean_squared_error(testY_1[0], testPredict_1[:,0]))  #testY_1, testX_1
print('Test_1 Score: %.5f RMSE' % (testScore_1))  
print('-----------------------------------------')

trainScore_2 = math.sqrt(mean_squared_error(trainY_2[0], trainPredict_2[:,0])) #trainY_2, trainX_2
print('Train_2 Score: %.5f RMSE' % (trainScore_2))  

testScore_2 = math.sqrt(mean_squared_error(testY_2[0], testPredict_2[:,0])) #testY_2, testX_2
print('Test_2 Score: %.5f RMSE' % (testScore_2)) 
print('=========================================')

trainScore_1_mse = mean_squared_error(trainY_1[0], trainPredict_1[:,0]) #trainY_1, trainX_1
print('Train_1 Score: %.5f MSE' % (trainScore_1_mse))  

testScore_1_mse = mean_squared_error(testY_1[0], testPredict_1[:,0])  #testY_1, testX_1
print('Test_1 Score: %.5f MSE' % (testScore_1_mse))  
print('-----------------------------------------')

trainScore_2_mse = mean_squared_error(trainY_2[0], trainPredict_2[:,0]) #trainY_2, trainX_2
print('Train_2 Score: %.5f MSE' % (trainScore_2_mse))  

testScore_2_mse = mean_squared_error(testY_2[0], testPredict_2[:,0]) #testY_2, testX_2
print('Test_2 Score: %.5f MSE' % (testScore_2_mse)) 
print('=========================================')

trainScore_1_mae = mean_absolute_error(trainY_1[0], trainPredict_1[:,0]) #trainY_1, trainX_1
print('Train_1 Score: %.5f MAE' % (trainScore_1_mae))  

testScore_1_mae = mean_absolute_error(testY_1[0], testPredict_1[:,0])  #testY_1, testX_1
print('Test_1 Score: %.5f MAE' % (testScore_1_mae))  
print('-----------------------------------------')

trainScore_2_mae = mean_absolute_error(trainY_2[0], trainPredict_2[:,0]) #trainY_2, trainX_2
print('Train_2 Score: %.5f MAE' % (trainScore_2_mae))  

testScore_2_mae = mean_absolute_error(testY_2[0], testPredict_2[:,0]) #testY_2, testX_2
print('Test_2 Score: %.5f MAE' % (testScore_2_mae)) 
print('=========================================')


# calculate mean, std, var of prediction  
trainPredict_1_mean = numpy.mean(trainPredict_1) # mean  
print('mean of trainPredict_1_mean == ', trainPredict_1_mean)
trainPredict_1_var = numpy.var(trainPredict_1)  # variance  
print('variance of trainPredict_1_var == ', trainPredict_1_var)
trainPredict_1_std = numpy.std(trainPredict_1) # standard deviation 
print('standard deviation of trainPredict_1_std == ', trainPredict_1_std)
print('-----------------------------------------')
trainPredict_2_mean = numpy.mean(trainPredict_2) # mean  
print('mean of trainPredict_2_mean == ', trainPredict_2_mean)
trainPredict_2_var = numpy.var(trainPredict_2)  # variance  
print('variance of trainPredict_2_var == ', trainPredict_2_var)
trainPredict_2_std = numpy.std(trainPredict_2) # standard deviation 
print('standard deviation of trainPredict_2_std == ', trainPredict_2_std)
print('-----------------------------------------')
testPredict_1_mean = numpy.mean(testPredict_1) # mean  
print('mean of testPredict_1_mean == ', testPredict_1_mean)
testPredict_1_var = numpy.var(testPredict_1)  # variance  
print('variance of testPredict_1_var == ', testPredict_1_var)
testPredict_1_std = numpy.std(testPredict_1) # standard deviation 
print('standard deviation of testPredict_1_std == ', testPredict_1_std)
print('-----------------------------------------')
testPredict_2_mean = numpy.mean(testPredict_2) # mean  
print('mean of testPredict_2_mean == ', testPredict_2_mean)
testPredict_2_var = numpy.var(testPredict_2)  # variance  
print('variance of testPredict_2_var == ', testPredict_2_var)
testPredict_2_std = numpy.std(testPredict_2) # standard deviation 
print('standard deviation of testPredict_2_std == ', testPredict_2_std)
print('=========================================')

# writing var, std of prediction  
file = open(title+"var_std_Prediction.txt", 'w') 
file.write(" # calculate mean, std, var of prediction \n")
file.write("mean of trainPredict_1_mean == \n")
file.write(str(trainPredict_1_mean))
file.write("variance of trainPredict_1_var == \n")
file.write(str(trainPredict_1_var))
file.write("standard deviation of trainPredict_1_std == \n")
file.write(str(trainPredict_1_std))
file.write("-----------------------------------------\n")
file.write("mean of trainPredict_2_mean == \n")
file.write(str(trainPredict_2_mean))
file.write("variance of trainPredict_2_var == \n")
file.write(str(trainPredict_2_var))
file.write("standard deviation of trainPredict_2_std == \n")
file.write(str(trainPredict_2_std))
file.write("-----------------------------------------\n")
file.write("mean of testPredict_1_mean == \n")
file.write(str(testPredict_1_mean))
file.write("variance of testPredict_1_var == \n")
file.write(str(testPredict_1_var))
file.write("standard deviation of testPredict_1_std == \n")
file.write(str(testPredict_1_std))
file.write("-----------------------------------------\n")
file.write("mean of testPredict_2_mean == \n")
file.write(str(testPredict_2_mean))
file.write("variance of testPredict_2_var == \n")
file.write(str(testPredict_2_var))
file.write("standard deviation of testPredict_2_std == \n")
file.write(str(testPredict_2_std))
file.write("-----------------------------------------\n")
file.close() 

# ---------------------------------
# shift train predictions for plotting

# --- trainX_1
trainPredictPlot_1 = numpy.empty_like(dataset_train_1)# train_1 = dataset_train_1[0:train_1_size,:] 
trainPredictPlot_1[:, :] = numpy.nan  
trainPredictPlot_1[timesteps:len(trainPredict_1)+timesteps, :] = trainPredict_1 #trainX_1
# --- trainX_2 
trainPredictPlot_2 = numpy.empty_like(dataset_train_2)# train_2 = dataset_train_2[0:train_2_size,:]
trainPredictPlot_2[:, :] = numpy.nan
trainPredictPlot_2[timesteps:len(trainPredict_2)+timesteps, :] = trainPredict_2 #trainX_1


# shift test predictions for plotting 
# --- testX_1
testPredictPlot_1 = numpy.empty_like(dataset_test_1)# test_1 = dataset_test_1[0:test_1_size,:]
testPredictPlot_1[:, :] = numpy.nan
testPredictPlot_1[timesteps:len(testPredict_1)+timesteps, :] = testPredict_1 #testX_1
# --- testX_2
testPredictPlot_2 = numpy.empty_like(dataset_test_2)# test_2 = dataset_test_2[0:test_2_size,:]
testPredictPlot_2[:, :] = numpy.nan
testPredictPlot_2[timesteps:len(testPredict_2)+timesteps, :] = testPredict_2 #testX_2

# ---------------------------------
# print to # Layer, Shape, Parameter 
print('model summary ... ', ensemble_model.summary())  # keras.models.model()
# ---------------------------------

s = title # head of file name 
# ---------------------------------
# plot graph
plot_model(ensemble_model, to_file=s+'model_image.png')  # from keras.utils import plot_model 
# ---------------------------------

# --- trainX_1
plt.plot(scaler1.inverse_transform(dataset_train_1), label='Groundtruth')
plt.plot(trainPredictPlot_1, label='Training')
plt.title('Train of Bearing3 X-axis from 12:17 to 14:07, 2hours (12sets)') 
plt.ylabel('Frequency')
plt.xlabel('X-axis samples')
plt.legend() 
plt.savefig(s+'Train_1_Xaxis.png') 
plt.show()
# --- trainX_2
plt.plot(scaler2.inverse_transform(dataset_train_2), label='Groundtruth')
plt.plot(trainPredictPlot_2, label='Training')
plt.title('Train of Bearing3 Y-axis from 12:17 to 14:07, 2hours (12sets)') 
plt.ylabel('Frequency')
plt.xlabel('Y-axis samples')
plt.legend() 
plt.savefig(s+'Train_2_Yaxis.png') 
plt.show()

# --- testX_1
plt.plot(scaler3.inverse_transform(dataset_test_1), label='Groundtruth')
plt.plot(testPredictPlot_1, label='Test')
plt.title('Test of Bearing 3 X-axis from 14:17 to 16:07, 2hours')   ### TEST bearing 3 
plt.ylabel('Frequency')
plt.xlabel('X-axis samples')
plt.legend() 
plt.savefig(s+'Test_1_Xaxis.png') 
plt.show()
# --- testX_2
plt.plot(scaler4.inverse_transform(dataset_test_2), label='Groundtruth')
plt.plot(testPredictPlot_2, label='Test')
plt.title('Test of Bearing 3 Y-axis from 14:17 to 16:07, 2hours')   ### TEST bearing 3 
plt.ylabel('Frequency')
plt.xlabel('Y-axis samples')
plt.legend() ## plt.plot(,label='')
plt.savefig(s+'Test_2_Yaxis.png')  
plt.show()
print('--------------------------------------------------------')   
print('history.___train___loss ===', history.loss)   
print('history.___test___loss ===', history.val_loss)

# model 1    
#print('history.first_output_model1_loss = ', history.first_output_model1_loss) 
#print('history.second_out_model1_loss = '  , history.second_output_model1_loss)

print('history.first_output_model1_mse =', history.first_output_model1_mse) 
print('history.first_output_model1_mae =', history.first_output_model1_mae)
print('history.first_output_model1_mape =', history.first_output_model1_mape) 
print('history.second_output_model1_mse =', history.second_output_model1_mse) 
print('histroy.second_output_model1_mae = ', history.second_output_model1_mae) 
print('history.second_output_model1_mape = ', history.second_output_model1_mape) 
        
#print('history.val_first_output_model1_loss = ', history.val_first_output_model1_loss)
#print('history.val_second_output_model1_loss = ', history.val_second_output_model1_loss) 

print('history.val_first_output_model1_mse = ', history.val_first_output_model1_mse)
print('history.val_first_output_model1_mae = ', history.val_first_output_model1_mae) 
print('history.val_first_output_model1_mape =', history.val_first_output_model1_mape) 
print('history.val_second_output_model1_mse = ', history.val_second_output_model1_mse) 
print('history.val_second_output_model1_mae = ', history.val_second_output_model1_mae) 
print('history.val_second_output_model1_mape = ', history.val_second_output_model1_mape)
print('--------------------------------------------------------')              

# model 2 
#print('history.first_output_model2_loss = ', history.first_output_model2_loss) 
#print('history.second_output_model2_loss = ', history.second_output_model2_loss)

print('history.first_output_model2_se = ', history.first_output_model2_mse)
print('history.first_output_model2_mae = ', history.first_output_model2_mae)
print('history.first_output_model2_mape = ', history.first_output_model2_mape)
print('history.second_output_model2_mse = ', history.second_output_model2_mse)
print('history.second_output_model2_mae = ', history.second_output_model2_mae)
print('history.second_output_model2_mape = ', history.second_output_model2_mape)

#print('history.val_first_output_model2_loss = ', history.val_first_output_model2_loss)
#print('history.val_second_output_model2_loss = ', history.val_second_output_model2_loss)

print('history.val_first_output_model2_mse =', history.val_first_output_model2_mse)
print('history.val_first_output_model2_mae = ', history.val_first_output_model2_mae)
print('history.val_first_output_model2_mape =', history.val_first_output_model2_mape)
print('history.val_second_output_model2_mse =', history.val_second_output_model2_mse)
print('history.val_second_output_model2_mae = ', history.val_second_output_model2_mae)
print('history.val_second_output_model2_mape = ', history.val_second_output_model2_mape)
print('--------------------------------------------------------')              

# model 3 
#print('history.first_output_model3_loss = ', history.first_output_model3_loss) 
#print('history.second_output_model3_loss = ', history.second_output_model3_loss)

print('history.first_output_model3_mse = ', history.first_output_model3_mse)
print('history.first_output_model3_mae = ', history.first_output_model3_mae)
print('history.first_output_model3_mape = ', history.first_output_model3_mape)
print('history.second_output_model3_mse = ', history.second_output_model3_mse)
print('history.second_output_model3_mae = ', history.second_output_model3_mae)
print('history.second_output_model3_mape = ', history.second_output_model3_mape)
        
#print('history.val_first_output_model3_loss = ', history.val_first_output_model3_loss)
#print('history.val_second_output_model3_loss = ', history.val_second_output_model3_loss)
                                          
print('history.val_first_output_model3_mse = ', history.val_first_output_model3_mse)
print('history.val_first_output_model3_mae = ', history.val_first_output_model3_mae)
print('history.val_first_output_model3_mape = ', history.val_first_output_model3_mape)
print('history.val_second_output_model3_mse = ', history.val_second_output_model3_mse)
print('history.val_second_output_model3_mae = ', history.val_second_output_model3_mae)
print('history.val_second_output_model3_mape = ', history.val_second_output_model3_mape)
print('--------------------------------------------------------')             

# model 4 
#print('history.first_output_model4_loss = ', history.first_output_model4_loss)
#print('history.second_output_model4_loss = ', history.second_output_model4_loss)
                                          
print('history.first_output_model4_mse = ', history.first_output_model4_mse)
print('history.first_output_model4_mae = ', history.first_output_model4_mae)
print('history.first_output_model4_mape = ', history.first_output_model4_mape)
print('history.second_output_model4_mse = ', history.second_output_model4_mse)
print('history.second_output_model4_mae = ', history.second_output_model4_mae)
print('history.second_output_model4_mape = ', history.second_output_model4_mape) 
            
#print('history.val_first_output_model4_loss = ', history.val_first_output_model4_loss)
#print('history.val_second_output_model4_loss = ', history.val_second_output_model4_loss)

print('history.val_first_output_model4_mse = ', history.val_first_output_model4_mse)
print('history.val_first_output_model4_mae = ', history.val_first_output_model4_mae)
print('history.val_first_output_model4_mape = ', history.val_first_output_model4_mape)
print('history.val_second_output_model4_mse = ', history.val_second_output_model4_mse)
print('history.val_second_output_model4_mae = ', history.val_second_output_model4_mae)
print('history.val_second_output_model4_mape = ', history.val_second_output_model4_mape)
print('--------------------------------------------------------')  


print('--------------------------------------------------------')  
plt.plot(history.loss, 'bo', label='Training loss (' + str(str(format(history.loss[-1],'.5f'))+')')) # Train loss = MSE 
#plt.plot(history.val_loss, 'b', label='test loss') # Test loss = MSE 
plt.plot(history.val_loss, 'b', label='Test loss (' + str(str(format(history.val_loss[-1],'.5f'))+')')) # Test loss = MSE 
plt.title('Ensemble Model loss of bearing 3 train and bearing 3 test')  ### TEST bearing 3
plt.xlabel('Epochs')
plt.ylabel('Loss (Mean Squared Error)')
plt.legend()  # plt.legend(['Train', 'Test']) # ==> label=..., label=...
plt.savefig(s+'Loss_model_mse.png')
plt.show()
print('--------------------------------------------------------')  

                                          
                                          
# loss (MSE) of model 1 
plt.plot(history.first_output_model1_mse, ':', label='x-axis Train loss (' + str(str(format(history.first_output_model1_mse[-31],'.5f'))+')')) # Train loss = MSE 
plt.plot(history.val_first_output_model1_mse, '--', label='x-axis Test loss (' + str(str(format(history.val_first_output_model1_mse[-31],'.5f'))+')')) # Test loss = MSE 
plt.plot(history.second_output_model1_mse, '-.', label='y-axis Train loss (' + str(str(format(history.second_output_model1_mse[-31],'.5f'))+')')) # Train loss = MSE 
plt.plot(history.val_second_output_model1_mse, '-', label='y-axis Test loss (' + str(str(format(history.val_second_output_model1_mse[-31],'.5f'))+')')) # Test loss = MSE 
plt.title('Model 1 Loss of bearing 3 train and bearing 3 test')       ### TEST bearing 3 
plt.xlabel('Epochs')
plt.ylabel('Loss (Mean Squared Error)')   #### Model 1 
plt.legend()  # plt.legend(['Train', 'Test']) # ==> label=..., label=...
plt.savefig(s+'Loss_MSE_Model_1.png')
plt.show() 

# loss (MSE) of model 2 
plt.plot(history.first_output_model2_mse, ':', label='x-axis Train loss (' + str(str(format(history.first_output_model2_mse[-21],'.5f'))+')')) # Train loss = MSE 
plt.plot(history.val_first_output_model2_mse, '--', label='x-axis Test loss (' + str(str(format(history.val_first_output_model2_mse[-21],'.5f'))+')')) # Test loss = MSE 
plt.plot(history.second_output_model2_mse, '-.', label='y-axis Train loss (' + str(str(format(history.second_output_model2_mse[-21],'.5f'))+')')) # Train loss = MSE 
plt.plot(history.val_second_output_model2_mse, '-', label='y-axis Test loss (' + str(str(format(history.val_second_output_model2_mse[-21],'.5f'))+')')) # Test loss = MSE 
plt.title('Model 2 Loss of bearing 3 train and bearing 3 test')   ### TEST bearing 3     
plt.xlabel('Epochs')
plt.ylabel('Loss (Mean Squared Error)') #### Model 2 
plt.legend()  # plt.legend(['Train', 'Test']) # ==> label=..., label=...
plt.savefig(s+'Loss_MSE_Model_2.png')
plt.show() 

# loss (MSE) of model 3 
plt.plot(history.first_output_model3_mse, ':', label='x-axis Train loss (' + str(str(format(history.first_output_model3_mse[-11],'.5f'))+')')) # Train loss = MSE 
plt.plot(history.val_first_output_model3_mse, '--', label='x-axis Test loss (' + str(str(format(history.val_first_output_model3_mse[-11],'.5f'))+')')) # Test loss = MSE 
plt.plot(history.second_output_model3_mse, '-.', label='y-axis Train loss (' + str(str(format(history.second_output_model3_mse[-11],'.5f'))+')')) # Train loss = MSE 
plt.plot(history.val_second_output_model3_mse, '-', label='y-axis Test loss (' + str(str(format(history.val_second_output_model3_mse[-11],'.5f'))+')')) # Test loss = MSE 
plt.title('Model 3 Loss of bearing 3 train and bearing 3 test') ### TEST bearing 3       
plt.xlabel('Epochs')
plt.ylabel('Loss (Mean Squared Error)')  #### Model 3 
plt.legend()  # plt.legend(['Train', 'Test']) # ==> label=..., label=...
plt.savefig(s+'Loss_MSE_Model_3.png')
plt.show() 
      
# loss (MSE) of model 4 
plt.plot(history.first_output_model4_mse, ':', label='x-axis Train loss (' + str(str(format(history.first_output_model4_mse[-1],'.5f'))+')')) # Train loss = MSE 
plt.plot(history.val_first_output_model4_mse, '--', label='x-axis Test loss (' + str(str(format(history.val_first_output_model4_mse[-1],'.5f'))+')')) # Test loss = MSE 
plt.plot(history.second_output_model4_mse, '-.', label='y-axis Train loss (' + str(str(format(history.second_output_model4_mse[-1],'.5f'))+')')) # Train loss = MSE 
plt.plot(history.val_second_output_model4_mse, '-', label='y-axis Test loss (' + str(str(format(history.val_second_output_model4_mse[-1],'.5f'))+')')) # Test loss = MSE 
plt.title('Model 4 Loss of bearing 3 train and bearing 3 test')   ### TEST bearing 3     
plt.xlabel('Epochs')
plt.ylabel('Loss (Mean Squared Error)')  #### Model 4 
plt.legend()  # plt.legend(['Train', 'Test']) # ==> label=..., label=...
plt.savefig(s+'Loss_MSE_Model_4.png')
plt.show() 

print('--------------------------------------------------------')  
      
# MAE of model 1 
plt.plot(history.first_output_model1_mae, ':', label='x-axis train MAE (' + str(str(format(history.first_output_model1_mae[-31],'.5f'))+')')) # Train loss = MSE 
plt.plot(history.val_first_output_model1_mae, '--', label='x-axis test MAE (' + str(str(format(history.val_first_output_model1_mae[-31],'.5f'))+')')) # Test loss = MSE 
plt.plot(history.second_output_model1_mae, '-.', label='y-axis train MAE (' + str(str(format(history.second_output_model1_mae[-31],'.5f'))+')')) # Train loss = MSE 
plt.plot(history.val_second_output_model1_mae, '-', label='y-axis test MAE (' + str(str(format(history.val_second_output_model1_mae[-31],'.5f'))+')')) # Test loss = MSE 
plt.title('Model 1 Loss of bearing 3 train and bearing 3 test')   ### TEST bearing 3
plt.xlabel('Epochs')
plt.ylabel('Loss (Mean Absolute Error)')   ##### Model 1
plt.legend()  # plt.legend(['Train', 'Test']) # ==> label=..., label=...
plt.savefig(s+'Loss_MAE_Model_1.png')
plt.show() 

# MAE of model 2 
plt.plot(history.first_output_model2_mae, ':', label='x-axis train MAE (' + str(str(format(history.first_output_model2_mae[-21],'.5f'))+')')) # Train loss = MSE 
plt.plot(history.val_first_output_model2_mae, '--', label='x-axis test MAE (' + str(str(format(history.val_first_output_model2_mae[-21],'.5f'))+')')) # Test loss = MSE 
plt.plot(history.second_output_model2_mae, '-.', label='y-axis train MAE (' + str(str(format(history.second_output_model2_mae[-21],'.5f'))+')')) # Train loss = MSE 
plt.plot(history.val_second_output_model2_mae, '-', label='y-axis test MAE (' + str(str(format(history.val_second_output_model2_mae[-21],'.5f'))+')')) # Test loss = MSE 
plt.title('Model 2 Loss of bearing 3 train and bearing 3 test')  ### TEST bearing 3
plt.xlabel('Epochs')
plt.ylabel('Loss (Mean Absolute Error)')   ##### Model 2
plt.legend()  # plt.legend(['Train', 'Test']) # ==> label=..., label=...
plt.savefig(s+'Loss_MAE_Model_2.png')
plt.show() 

# MAE of model 3 
plt.plot(history.first_output_model3_mae, ':', label='x-axis train MAE (' + str(str(format(history.first_output_model3_mae[-11],'.5f'))+')')) # Train loss = MSE 
plt.plot(history.val_first_output_model3_mae, '--', label='x-axis test MAE (' + str(str(format(history.val_first_output_model3_mae[-11],'.5f'))+')')) # Test loss = MSE 
plt.plot(history.second_output_model3_mae, '-.', label='y-axis train MAE (' + str(str(format(history.second_output_model3_mae[-11],'.5f'))+')')) # Train loss = MSE 
plt.plot(history.val_second_output_model3_mae, '-', label='y-axis test MAE (' + str(str(format(history.val_second_output_model3_mae[-11],'.5f'))+')')) # Test loss = MSE 
plt.title('Model 3 Loss of bearing 3 train and bearing 3 test')  ### TEST bearing 3
plt.xlabel('Epochs')
plt.ylabel('Loss (Mean Absolute Error)')   ##### Model 3
plt.legend()  # plt.legend(['Train', 'Test']) # ==> label=..., label=...
plt.savefig(s+'Loss_MAE_Model_3.png')
plt.show() 

# MAE of model 4 
plt.plot(history.first_output_model4_mae, ':', label='x-axis train MAE (' + str(str(format(history.first_output_model4_mae[-1],'.5f'))+')')) # Train loss = MSE 
plt.plot(history.val_first_output_model4_mae, '--', label='x-axis test MAE (' + str(str(format(history.val_first_output_model4_mae[-1],'.5f'))+')')) # Test loss = MSE 
plt.plot(history.second_output_model4_mae, '-.', label='y-axis train MAE (' + str(str(format(history.second_output_model4_mae[-1],'.5f'))+')')) # Train loss = MSE 
plt.plot(history.val_second_output_model4_mae, '-', label='y-axis test MAE (' + str(str(format(history.val_second_output_model4_mae[-1],'.5f'))+')')) # Test loss = MSE 
plt.title('Model 4 Loss of bearing 3 train and bearing 3 test')  ### TEST bearing 3
plt.xlabel('Epochs')
plt.ylabel('Loss (Mean Absolute Error)')  ##### Model 4
plt.legend()  # plt.legend(['Train', 'Test']) # ==> label=..., label=...
plt.savefig(s+'Loss_MAE_Model_4.png')
plt.show() 
      
print('--------------------------------------------------------') 
      
# MAPE of model 1 
plt.plot(history.first_output_model1_mape, ':', label='x-axis train MAPE (' + str(str(format(history.first_output_model1_mape[-31],'.5f'))+')')) # Train loss = MSE 
plt.plot(history.val_first_output_model1_mape, '--', label='x-axis test MAPE (' + str(str(format(history.val_first_output_model1_mape[-31],'.5f'))+')')) # Test loss = MSE 
plt.plot(history.second_output_model1_mape, '-.', label='y-axis train MAPE (' + str(str(format(history.second_output_model1_mape[-31],'.5f'))+')')) # Train loss = MSE 
plt.plot(history.val_second_output_model1_mape, '-', label='y-axis test MAPE (' + str(str(format(history.val_second_output_model1_mape[-31],'.5f'))+')')) # Test loss = MSE 
plt.title('Model 1 Loss of bearing 3 train and bearing 3 test')  ### TEST bearing 3
plt.xlabel('Epochs')
plt.ylabel('Loss (Mean Absolute Percentage Error)') ##### Model 1
plt.legend()  # plt.legend(['Train', 'Test']) # => label=..., label=...
plt.savefig(s+'Loss_MAPE_Modle_1.png')
plt.show() 

# MAPE of model 2 
plt.plot(history.first_output_model2_mape, ':', label='x-axis train MAPE (' + str(str(format(history.first_output_model2_mape[-21],'.5f'))+')')) # Train loss = MSE 
plt.plot(history.val_first_output_model2_mape, '--', label='x-axis test MAPE (' + str(str(format(history.val_first_output_model2_mape[-21],'.5f'))+')')) # Test loss = MSE 
plt.plot(history.second_output_model2_mape, '-.', label='y-axis train MAPE (' + str(str(format(history.second_output_model2_mape[-21],'.5f'))+')')) # Train loss = MSE 
plt.plot(history.val_second_output_model2_mape, '-', label='y-axis test MAPE (' + str(str(format(history.val_second_output_model2_mape[-21],'.5f'))+')')) # Test loss = MSE 
plt.title('Model 2 Loss of bearing 3 train and bearing 3 test')  ### TEST bearing 3
plt.xlabel('Epochs')
plt.ylabel('Loss (Mean Absolute Percentage Error)') ##### Model 2
plt.legend()  # plt.legend(['Train', 'Test']) # => label=..., label=...
plt.savefig(s+'Loss_MAPE_Modle_2.png')
plt.show() 

# MAPE of model 3 
plt.plot(history.first_output_model3_mape, ':', label='x-axis train MAPE (' + str(str(format(history.first_output_model3_mape[-11],'.5f'))+')')) # Train loss = MSE 
plt.plot(history.val_first_output_model3_mape, '--', label='x-axis test MAPE (' + str(str(format(history.val_first_output_model3_mape[-11],'.5f'))+')')) # Test loss = MSE 
plt.plot(history.second_output_model3_mape, '-.', label='y-axis train MAPE (' + str(str(format(history.second_output_model3_mape[-11],'.5f'))+')')) # Train loss = MSE 
plt.plot(history.val_second_output_model3_mape, '-', label='y-axis test MAPE (' + str(str(format(history.val_second_output_model3_mape[-11],'.5f'))+')')) # Test loss = MSE 
plt.title('Model 3 Loss of bearing 3 train and bearing 3 test') ### TEST bearing 3
plt.xlabel('Epochs')
plt.ylabel('Loss (Mean Absolute Percentage Error)') ##### Model 3
plt.legend()  # plt.legend(['Train', 'Test']) # => label=..., label=...
plt.savefig(s+'Loss_MAPE_Modle_3.png')
plt.show() 

# MAPE of model 4 
plt.plot(history.first_output_model4_mape, ':', label='x-axis train MAPE (' + str(str(format(history.first_output_model4_mape[-1],'.5f'))+')')) # Train loss = MSE 
plt.plot(history.val_first_output_model4_mape, '--', label='x-axis test MAPE (' + str(str(format(history.val_first_output_model4_mape[-1],'.5f'))+')')) # Test loss = MSE 
plt.plot(history.second_output_model4_mape, '-.', label='y-axis train MAPE (' + str(str(format(history.second_output_model4_mape[-1],'.5f'))+')')) # Train loss = MSE 
plt.plot(history.val_second_output_model4_mape, '-', label='y-axis test MAPE (' + str(str(format(history.val_second_output_model4_mape[-1],'.5f'))+')')) # Test loss = MSE 
plt.title('Model 1 Loss of bearing 3 train and bearing 3 test')  ### TEST bearing 3
plt.xlabel('Epochs')
plt.ylabel('Loss (Mean Absolute Percentage Error)') ##### Model 4
plt.legend()  # plt.legend(['Train', 'Test']) # => label=..., label=...
plt.savefig(s+'Loss_MAPE_Modle_4.png')
plt.show()     
print('--------------------------------------------------------') 

# ---------------- density plots 
plt.subplot(221)
series1 = df_train_1 #df_train_1 = pandas.read_csv(t, header=None, sep='	', usecols=[4], engine='python') 
series1.plot(kind='kde') 
plt.title("Density Plots of Train X-axis")

plt.subplot(222)
series2 = df_train_2 #df_train_2 = pandas.read_csv(t, header=None, sep='	', usecols=[5], engine='python')  
series2.plot(kind='kde')
plt.title("Density Plots of Train Y-axis")

plt.subplot(223)   # name 'df_test_1' is not defined 
series3 = df_test_1 #df_test_1 = pandas.read_csv(t1, header=None, sep='	', usecols=[4], engine='python')
series3.plot(kind='kde')
plt.title("Density Plots of Test X-axis")

plt.subplot(224)
series4 = df_test_2 #df_test_2 = pandas.read_csv(t1, header=None, sep='	', usecols=[5], engine='python')
series4.plot(kind='kde')
plt.title("Density Plots of Test Y-axis")

plt.tight_layout()
plt.savefig(s+'densityplots.png') 
plt.show()

# ---------------- Train Boxplot 
plt.subplot(221)
df_train_x = pandas.DataFrame(df_train_1) #TrainX_1 dataset 
df_train_x.boxplot() 
plt.title("X-axis train target dataset")

plt.subplot(222)
df_trainPredictPlot_1 = pandas.DataFrame(trainPredictPlot_1) #TrainX_1 Prediction
df_trainPredictPlot_1.boxplot()  
plt.title("X-axis train prediction")

plt.subplot(223)
df_train_y = pandas.DataFrame(df_train_2) #TrainX_2 dataset
df_train_y.boxplot() 
plt.title("Y-axis train target dataset")

plt.subplot(224)
df_trainPredictPlot_2 = pandas.DataFrame(trainPredictPlot_2) #TrainX_2  Prediction
df_trainPredictPlot_2.boxplot()  
plt.title("Y-axis train prediction")
# -------
plt.tight_layout()
plt.savefig(s+'boxplot_train.png') 
plt.show()
# ---------------- Test Boxplot 
plt.subplot(221) 
df_test_x = pandas.DataFrame(df_test_1) #Test dataset
df_test_x.boxplot() 
plt.title("X-axis test target dataset")

plt.subplot(222)
df_testPredictPlot_1 = pandas.DataFrame(testPredictPlot_1) # TestX_1  
df_testPredictPlot_1.boxplot()
plt.title("X-axis test prediction")

plt.subplot(223)
df_test_y = pandas.DataFrame(df_test_2) #Test dataset
df_test_y.boxplot() 
plt.title("Y-axis test target dataset")

plt.subplot(224)
df_testPredictPlot_2 = pandas.DataFrame(testPredictPlot_2)  # TestX_2
df_testPredictPlot_2.boxplot()
plt.title("Y-axis test prediction")
# -------
plt.tight_layout()
plt.savefig(s+'boxplot_test.png') 
plt.show()
# ---------------------------------
print('------- Train_1_x_dataframe.describe() -------')
print(df_train_x.describe())
print('------- df_trainPredictPlot_1_x.describe() -------')
print(df_trainPredictPlot_1.describe())
print('------- Train_2_y_dataframe.describe() -------')
print(df_train_x.describe())
print('------- df_trainPredictPlot_2_y.describe() -------')  
print(df_trainPredictPlot_2.describe())
# ---------------------------------
print('------- Test 1 x.describe() -------')
print(df_test_x.describe())
print('------- df_testPredictPlot_1_x.describe() -------')
print(df_testPredictPlot_1.describe())
print('------- Test 2 y.describe() -------')
print(df_test_y.describe())
print('------- df_testPredictPlot_2_y.describe() -------')
print(df_testPredictPlot_2.describe())
# ---------------------------------
print('--- end of codes')

Using TensorFlow backend.


 dataset_train_1.shape ==  (245760, 1)
 dataset_train_2.shape ==  (245760, 1)
 dataset_test_1.shape ==  (245760, 1)
 dataset_test_2.shape ==  (245760, 1)
-----------------------------------------
 dataset_train_1.shape ==  (245760, 1)
 dataset_train_2.shape ==  (245760, 1)
 dataset_test_1.shape ==  (245760, 1)
 dataset_test_2.shape ==  (245760, 1)
-----------------------------------------
mean of dataset_train_1 ==  0.45230213
variance of dataset_train_1 ==  0.0010688052
standard deviation of dataset_train_1 ==  0.032692585
-----------------------------------------
mean of dataset_train_1 ==  0.5091193
variance of dataset_train_2 ==  0.0024536815
standard deviation of dataset_train_2 ==  0.04953465
-----------------------------------------
mean of dataset_test_1 ==  0.5687847
variance of dataset_test_1 ==  0.0011410454
standard deviation of dataset_test_1 ==  0.033779364
-----------------------------------------
mean of dataset_test_2 ==  0.46432123
variance of dataset_test_2 ==  0.002

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

...

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 4 Axes>

<Figure size 640x480 with 4 Axes>

------- Train_1_x_dataframe.describe() -------
                   4
count  245760.000000
mean       -0.114734
std         0.225318
min        -3.232000
25%        -0.247000
50%        -0.112000
75%         0.020000
max         3.660000
------- df_trainPredictPlot_1_x.describe() -------
                   0
count  245730.000000
mean       -0.113342
std         0.006012
min        -0.188977
25%        -0.116848
50%        -0.113177
75%        -0.109616
max        -0.049243
------- Train_2_y_dataframe.describe() -------
                   4
count  245760.000000
mean       -0.114734
std         0.225318
min        -3.232000
25%        -0.247000
50%        -0.112000
75%         0.020000
max         3.660000
------- df_trainPredictPlot_2_y.describe() -------
                   0
count  245730.000000
mean       -0.120076
std         0.011624
min        -0.285542
25%        -0.126959
50%        -0.119963
75%        -0.113054
max         0.030164
------- Test 1 x.describe() -------
            